In [1]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import load_model
from sklearn.model_selection import train_test_split
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import io
from skimage.color import rgba2rgb
from skimage.color import gray2rgb
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet_v2 import MobileNetV2


Using TensorFlow backend.


In [19]:
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return 1-f1_val

def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
os.chdir('gdrive/My Drive/Skyhacks 2020/SkyHacks2020/skyhacks_hackathon_dataset')

In [7]:
os.listdir()

['training_labels.csv',
 'skyhacks_hackathon_dataset.zip',
 'skyhacks_hackathon_dataset']

In [5]:
train_labels = pd.read_csv('training_labels.csv')

In [6]:
os.chdir('training_images')

In [10]:
input_shape = (96,96,3)

In [8]:
train_labels.set_index('Name',inplace=True)

In [11]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
model_Mobile1 = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
model_Mobile2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
model_Resnet1 = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
model_Resnet2 = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)

In [2]:
model_Densenet1 = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
model_Densenet2 = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

NameError: name 'DenseNet121' is not defined

In [ ]:
small_object = ['Animals','Bench','Cross', 'Food', 'Footpath', 'Furniture', 'Monument','Motor vehicle','Person','Plants','Road', 'Rocks','Stairs','Trees','Windows']
train_labels_SO = train_labels[small_object]


In [3]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
 
# Driver Code
li1 = [10, 15, 20, 25, 30, 35, 40]
li2 = [25, 40, 35]
print(Diff(li1, li2))

ValueError: list.remove(x): x not in list

In [ ]:
big_object = Diff(train_labels.columns, small_object)
train_labels_SO = train_labels[big_object]

In [12]:
def final_model(premodel, shape):
    x = premodel.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    stuff = Dense(shape, activation='sigmoid')(x)

    model = Model(inputs=premodel.input, outputs=[stuff])

    for layer in premodel.layers:
        layer.trainable = False
    return (model)

In [13]:
labels = train_labels.columns
targets = train_labels.to_numpy()

In [14]:
imgset = []

for name in train_labels.index:
    try:
      image = io.imread(name)/255
    except:
      try:
        image = io.imread(name.lower())/255
      except:
        try:
          image = io.imread(name.upper())/255
        except:
          pass
    
    image = resize(image,input_shape)
    imgset.append(image)
    if len(imgset)%500 == 0:
      print(len(imgset))
    
imgset = np.array([np.array(imgset[i]) for i in range(len(imgset))])

500
1000
1500
2000
2500
3000
3500
4000


In [16]:
X_train, X_test, y_train, y_test = train_test_split(imgset, targets, test_size=0.2, random_state = 42)

In [18]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

In [22]:
model.losses

[]

In [23]:
EarlyStop = EarlyStopping(monitor= "val_dense_6_f1", patience = 5, restore_best_weights=True)

losses = {
	"dense_1": "categorical_crossentropy",
}

model.compile(optimizer='adam', loss=losses, metrics = [f1])

In [24]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=12, callbacks = [EarlyStop], batch_size = 16)  

Epoch 1/12
208/208 [==============================] - 30s 142ms/step - loss: 11.0220 - f1: 0.4676 - val_loss: 10.7223 - val_f1: 0.4208
Epoch 2/12
208/208 [==============================] - 31s 149ms/step - loss: 9.5533 - f1: 0.3518 - val_loss: 10.8150 - val_f1: 0.4079
Epoch 3/12
208/208 [==============================] - 29s 138ms/step - loss: 8.7707 - f1: 0.3066 - val_loss: 10.7350 - val_f1: 0.4034
Epoch 4/12
208/208 [==============================] - 29s 137ms/step - loss: 8.1546 - f1: 0.2639 - val_loss: 11.1804 - val_f1: 0.4081
Epoch 5/12
208/208 [==============================] - 29s 139ms/step - loss: 7.6886 - f1: 0.2187 - val_loss: 11.5369 - val_f1: 0.4181
Epoch 6/12
208/208 [==============================] - 29s 137ms/step - loss: 7.3168 - f1: 0.1792 - val_loss: 11.8983 - val_f1: 0.4151
Epoch 7/12
208/208 [==============================] - 29s 138ms/step - loss: 7.0921 - f1: 0.1455 - val_loss: 12.3914 - val_f1: 0.4217
Epoch 8/12
208/208 [==============================] - 29s 138

In [26]:
model.save(os.getcwd())

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Skyhacks 2020/SkyHacks2020/skyhacks_hackathon_dataset/training_images/assets


In [ ]:
model.